# Aggregated & Preprocessed Traffic Data — EDA, Cleaning & Feature Engineering

**What this notebook does (end-to-end)**

1. Loads raw traffic data (CSV(s)) and optional events/metadata files.  
2. Cleans data: handles missing values, duplicates, and corrects dtypes.  
3. Aggregates traffic counts to hourly intervals per junction.  
4. Preprocesses numeric columns (scaling/normalizing) and encodes categorical/time features.  
5. Engineers time-based features, lag features, rolling windows, weekend/event flags.  
6. Performs feature relevance analysis (correlation, RandomForest importance).  
7. Saves the final aggregated & preprocessed dataset for modeling/submission.

**How to use**
- Place your raw CSV(s) in a folder `data/`. Typical expected columns in traffic CSV:
  - `timestamp` (or `datetime`) — string parsable by `pd.to_datetime`
  - `junction_id` (or similar) — identifier for junction/intersection
  - `vehicle_count` (or `count`, `vehicles`) — numeric count
  - optionally `vehicle_type`, `lane_id`, etc.

- If you have a special events file, name it `events.csv` with at least:
  - `date` or `start_date`/`end_date` and `event_name` or `is_event` flag.

- Run the notebook, inspect outputs, adjust column names in the "CONFIG" cell to match your dataset.



## Requirements

This notebook uses common Python data libraries:
- pandas, numpy, matplotlib, scikit-learn
- (optional) statsmodels for time series diagnostics, but not required

Make sure these libraries are installed in your environment. The notebook is written to be robust if a few optional packages are missing.



In [ ]:
# -----------------------
# CONFIG: Edit these to match your files/columns
# -----------------------
RAW_DATA_DIR = "data"                      # folder containing raw CSVs
RAW_FILES = ["traffic_raw.csv"]            # list of CSV filenames to load (order doesn't matter)
EVENTS_FILE = "events.csv"                 # optional special events file (set to None if not available)

# Column name mapping: rename your dataset's columns to these canonical names after load
COL_TIMESTAMP = "timestamp"      # e.g. "timestamp" or "datetime"
COL_JUNCTION = "junction_id"     # identifier for junction/intersection
COL_COUNT = "vehicle_count"      # numeric vehicle count column name

# Output filenames
AGG_OUT_CSV = "aggregated_preprocessed.csv"

# Aggregation settings
RESAMPLE_FREQ = "H"   # hourly aggregation
TIMEZONE = None       # e.g. "Asia/Kolkata" if timestamps are localized; set to None if naive


In [ ]:
# Imports
import os
import glob
from datetime import timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error

# plt style: we will use matplotlib only (no seaborn) and won't force specific colors.
# For notebooks, enable inline plotting (if using jupyter).
try:
    get_ipython().run_line_magic('matplotlib', 'inline')
except Exception:
    pass


In [ ]:
# Utility functions reused throughout the notebook

def load_and_concat_csvs(folder, file_list):
    """Load CSVs specified in file_list located in folder, concat into one DataFrame."""
    dfs = []
    for fn in file_list:
        path = os.path.join(folder, fn)
        if not os.path.exists(path):
            print(f"Warning: {path} not found — skipping.")
            continue
        df = pd.read_csv(path)
        df['_source_file'] = fn
        dfs.append(df)
    if len(dfs) == 0:
        raise FileNotFoundError("No data files found. Put your CSV(s) in the data/ directory and update RAW_FILES.")
    combined = pd.concat(dfs, axis=0, ignore_index=True)
    return combined

def safe_to_datetime(df, col, tz=None):
    df[col] = pd.to_datetime(df[col], errors='coerce')
    if tz is not None:
        # Convert naive timestamps to the desired timezone (assumes input is naive/local)
        df[col] = df[col].dt.tz_localize(tz, ambiguous='infer', nonexistent='shift_forward')
    return df

def print_df_info(df, name='df'):
    print(f"=== {name} shape: {df.shape} ===")
    print(df.dtypes)
    print(df.head(3))
    print('\nMissing values per column:\n', df.isna().sum())


In [ ]:
# Load raw data
raw = load_and_concat_csvs(RAW_DATA_DIR, RAW_FILES)
print('Loaded rows:', len(raw))
print_df_info(raw, 'raw')

In [ ]:
# Standardize column names
raw_cols = {c.lower(): c for c in raw.columns}  # quick mapping
# Rename to canonical names if possible (reasonable heuristics)
rename_map = {}
for c in raw.columns:
    lc = c.lower()
    if 'time' in lc or 'date' in lc:
        rename_map[c] = COL_TIMESTAMP
    elif 'junction' in lc or 'intersection' in lc:
        rename_map[c] = COL_JUNCTION
    elif 'count' in lc or 'vehicle' in lc or 'vehicles' in lc:
        # prefer a name containing 'count'
        rename_map[c] = COL_COUNT

if rename_map:
    raw = raw.rename(columns=rename_map)
    print('Columns renamed using heuristics:', rename_map)
else:
    print('No heuristic renames applied; ensure your columns match the CONFIG variables.')

# Make sure canonical columns are present
missing = [COL_TIMESTAMP, COL_JUNCTION, COL_COUNT]
for m in missing:
    if m not in raw.columns:
        print(f"WARNING: expected column '{m}' not found. Please update CONFIG or column names.")

In [ ]:
# Parse timestamps
raw = safe_to_datetime(raw, COL_TIMESTAMP, tz=TIMEZONE)
print('After parsing timestamps:')
print_df_info(raw, 'raw')

# Drop rows where timestamp or junction is missing — they cannot be used for aggregation
raw = raw.dropna(subset=[COL_TIMESTAMP, COL_JUNCTION])

# Ensure counts numeric
raw[COL_COUNT] = pd.to_numeric(raw[COL_COUNT], errors='coerce')

# Basic duplicates handling
pre_dup = len(raw)
raw = raw.drop_duplicates()
print(f"Dropped {pre_dup - len(raw)} duplicate rows.")

In [ ]:
# Aggregate to hourly intervals per junction
# Step: set timestamp as index for resampling convenience
raw = raw.set_index(COL_TIMESTAMP)

# If there are other columns (e.g., vehicle_type) you may want to pivot or aggregate them separately.
agg = (raw
       .groupby([pd.Grouper(freq=RESAMPLE_FREQ), COL_JUNCTION])
       [COL_COUNT]
       .sum()
       .reset_index()
       .rename(columns={COL_COUNT: 'vehicle_count_hourly'}))

# Ensure continuous hourly grid per junction (fill missing hours with 0 or NaN depending on preference).
# We'll create a full grid and left-join to guarantee consistent time rows for each junction.
junctions = agg[COL_JUNCTION].unique()
full_frames = []
for j in junctions:
    jdf = agg[agg[COL_JUNCTION] == j].set_index(COL_TIMESTAMP).sort_index()
    idx = pd.date_range(start=jdf.index.min(), end=jdf.index.max(), freq=RESAMPLE_FREQ)
    jdf = jdf.reindex(idx)
    jdf[COL_JUNCTION] = j
    jdf.index.name = COL_TIMESTAMP
    full_frames.append(jdf.reset_index())

agg_full = pd.concat(full_frames, ignore_index=True)
# Missing vehicle counts likely mean 0 (no vehicles recorded) or missing sensor -- choose your domain choice.
# We'll impute missing hourly counts with 0 (comment: change to .fillna(method='ffill') if sensor gaps)
agg_full['vehicle_count_hourly'] = agg_full['vehicle_count_hourly'].fillna(0)
print('Aggregated shape (hourly per junction):', agg_full.shape)
agg_full.head()

In [ ]:
# Time-based features
df = agg_full.copy()
df[COL_TIMESTAMP] = pd.to_datetime(df[COL_TIMESTAMP])  # ensure column exists
df['hour'] = df[COL_TIMESTAMP].dt.hour
df['day_of_week'] = df[COL_TIMESTAMP].dt.dayofweek      # Monday=0
df['weekday_name'] = df[COL_TIMESTAMP].dt.day_name()
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
df['month'] = df[COL_TIMESTAMP].dt.month

# cyclical encoding of hour (useful for many models)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

print_df_info(df, 'df (after time features)')
df.head()

In [ ]:
# Lag and rolling features
def add_lag_features(df, group_col, value_col, lags=[1,2,3,24,48], rolling_windows=[3,6,24]):
    df = df.sort_values([group_col, COL_TIMESTAMP]).reset_index(drop=True)
    for lag in lags:
        col_name = f'{value_col}_lag_{lag}'
        df[col_name] = df.groupby(group_col)[value_col].shift(lag)
    for w in rolling_windows:
        col_name = f'{value_col}_rollmean_{w}'
        df[col_name] = df.groupby(group_col)[value_col].shift(1).rolling(w).mean().reset_index(level=0, drop=True)
    return df

df = add_lag_features(df, COL_JUNCTION, 'vehicle_count_hourly', lags=[1,2,3,24], rolling_windows=[3,6,24])
# Replace NaNs from lag features with 0 (or keep NaN if you prefer explicit missingness)
lag_cols = [c for c in df.columns if 'lag' in c or 'rollmean' in c]
df[lag_cols] = df[lag_cols].fillna(0)
df.head()

In [ ]:
# Merge special events (if provided)
events_path = os.path.join(RAW_DATA_DIR, EVENTS_FILE) if EVENTS_FILE else None
if events_path and os.path.exists(events_path):
    events = pd.read_csv(events_path)
    # Expected minimal structure: either a single date column or start_date/end_date
    # Normalize events to a per-date flag
    if 'date' in events.columns:
        events['date'] = pd.to_datetime(events['date']).dt.date
        event_dates = set(events['date'].tolist())
        df['is_event'] = df[COL_TIMESTAMP].dt.date.isin(event_dates).astype(int)
    elif 'start_date' in events.columns and 'end_date' in events.columns:
        events['start_date'] = pd.to_datetime(events['start_date']).dt.date
        events['end_date'] = pd.to_datetime(events['end_date']).dt.date
        def check_event(d):
            for _, r in events.iterrows():
                if r['start_date'] <= d <= r['end_date']:
                    return 1
            return 0
        df['is_event'] = df[COL_TIMESTAMP].dt.date.apply(check_event)
    else:
        print('events.csv found but format not recognized. Expected `date` or `start_date`/`end_date`.')
else:
    df['is_event'] = 0
    print('No events file found; is_event = 0 for all rows.')
df['is_event'] = df['is_event'].fillna(0).astype(int)
df.head()

In [ ]:
# Simple visualizations — one per plot as recommended

# 1) Distribution of hourly vehicle counts (overall)
plt.figure(figsize=(8,4))
plt.hist(df['vehicle_count_hourly'].clip(upper=df['vehicle_count_hourly'].quantile(0.99)), bins=50)
plt.title('Distribution of hourly vehicle counts (capped at 99th pct)')
plt.xlabel('vehicle_count_hourly')
plt.ylabel('frequency')
plt.show()

# 2) Time-series for a sample junction (first junction)
sample_junc = df[COL_JUNCTION].unique()[0]
sample_ts = df[df[COL_JUNCTION] == sample_junc].set_index(COL_TIMESTAMP)['vehicle_count_hourly']
plt.figure(figsize=(12,4))
plt.plot(sample_ts.index, sample_ts.values)
plt.title(f'Hourly vehicle count — junction {sample_junc}')
plt.xlabel('time')
plt.ylabel('vehicle_count_hourly')
plt.show()

# 3) Average hourly profile (mean per hour of day)
hourly_profile = df.groupby('hour')['vehicle_count_hourly'].mean()
plt.figure(figsize=(8,4))
plt.plot(hourly_profile.index, hourly_profile.values, marker='o')
plt.title('Average traffic profile by hour of day (all junctions)')
plt.xlabel('hour (0-23)')
plt.ylabel('mean vehicle_count_hourly')
plt.xticks(range(0,24))
plt.show()

# 4) Boxplot of counts by weekday
plt.figure(figsize=(10,5))
# create list of arrays per weekday for plotting
weekday_data = [df[df['day_of_week']==i]['vehicle_count_hourly'].values for i in range(7)]
plt.boxplot(weekday_data, labels=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], showfliers=False)
plt.title('Hourly counts distribution by weekday (no outliers)')
plt.ylabel('vehicle_count_hourly')
plt.show()

In [ ]:
# Scaling/Normalization example
numeric_cols = ['vehicle_count_hourly'] + [c for c in lag_cols if c in df.columns] + ['hour_sin', 'hour_cos']
numeric_cols = [c for c in numeric_cols if c in df.columns]

scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])

print('Scaled numeric columns with StandardScaler. Sample:')
df_scaled[numeric_cols].head()

In [ ]:
# Feature selection: correlation matrix
corr_cols = ['vehicle_count_hourly'] + [c for c in df_scaled.columns if c not in [COL_TIMESTAMP, COL_JUNCTION, 'weekday_name']]
corr = df_scaled[corr_cols].corr()
print('Top correlations with target:')
print(corr['vehicle_count_hourly'].abs().sort_values(ascending=False).head(10))

# RandomForest feature importance (quick demonstration)
X = df_scaled.drop(columns=[COL_TIMESTAMP, COL_JUNCTION, 'vehicle_count_hourly', 'weekday_name'])
y = df_scaled['vehicle_count_hourly']
# Simple train-test split for importance estimation
X_train, X_test, y_train, y_test = train_test_split(X.fillna(0), y.fillna(0), test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
imp = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print('\nRandomForest importances (top 20):')
print(imp.head(20))

# Optional: keep top K features
top_k = imp.head(20).index.tolist()
X_top = X[top_k]

In [ ]:
# Save final aggregated & preprocessed CSV for submission
out_path = os.path.join('.', 'aggregated_preprocessed.csv')
# Choose df or df_scaled as your final output. We'll save both.
df.to_csv(out_path.replace('.csv', '_rawfeatures.csv'), index=False)
df_scaled.to_csv(out_path.replace('.csv', '_scaled.csv'), index=False)
print('Saved files:', out_path.replace('.csv', '_rawfeatures.csv'), 'and', out_path.replace('.csv', '_scaled.csv'))

In [ ]:
# Quick baseline model RMSE on test set (using top features)
y_pred = rf.predict(X_test.fillna(0))
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Baseline RandomForest RMSE (on scaled target): {rmse:.4f}')
# Note: target was scaled; if you want RMSE in original units, inverse-transform the target scaling accordingly.


## Next steps & submission tips

- Inspect the saved files `*_rawfeatures.csv` and `*_scaled.csv`. These are ready for model training.  
- If your assignment requires a single file under 10 MB, compress the CSV or choose the scaled/raw file as appropriate.  
- Update the `CONFIG` cell at the top to match exact column names in your dataset before running.  
- For production modeling, consider:
  - More carefully handling missing sensor intervals vs true zeros.
  - Encoding categorical features (junction metadata) with embeddings or target encoding.
  - Using cross-validation over time (time-series split) rather than random train-test split.
  - Feature selection using time-series-aware validation and permutation importance.

**Deliverables for submission (recommended):**
1. `Aggregated_Preprocessed_Data_Karthik.ipynb` — this notebook with code, comments, and plots.
2. `aggregated_preprocessed_rawfeatures.csv` — aggregated with engineered features.
3. `aggregated_preprocessed_scaled.csv` — scaled numeric features for direct model training.
4. Short presentation (PDF/PowerPoint) summarizing EDA findings and peak-hour insights.

Good luck — if you want, I can further tailor the notebook to your exact column names or add specific visualizations (heatmap of correlations, interactive plots) — tell me your raw CSV header and I'll adapt the notebook automatically.
